In [ ]:
using StatsBase: mean

# Graph
using Graphs: SimpleGraph, connected_components
using SimpleWeightedGraphs: SimpleWeightedGraph

# Plot
using Plots: plot
using Colors: distinguishable_colors, @colorant_str
using GraphPlot: gplot, spring_layout
using Compose

# Original modules
include("../src/Simulation.jl")
include("../src/SimPlot.jl")
using .Simulation
using .SimPlot

In [ ]:
param = Param(
    initial_N = 100,
    initial_graph_weight = 0.2,
    δ = 0.1,
    β = 0.1,
    σ = 0.1,
    generations = 1_000,
)
df, graph_weights = Simulation.run(param)
SimPlot.plot_output_df(df)

In [ ]:
param = Param(
    initial_N = 100,
    initial_graph_weight = 0.2,
    δ = 0.1,
    β = 0.1,
    σ = 0.1,
    μ = 0.01,
    generations = 1000,
)

model = Model(param)
model.env_severity_vec = repeat([1.0], Int(model.param.initial_N))
model.strategy_vec = rand(model.param.rng, [C, D], model.param.initial_N)

output_df = Simulation.make_output_df(model.param)

set_default_graphic_size(24cm, 8cm)

strategy_vec_vec = []
graph_weights_vec = []

for generation = 1:model.param.generations
    interaction!(model)
    death_and_birth!(model, generation)
    log!(output_df, generation, model)
    push!(strategy_vec_vec, copy(model.strategy_vec))
    push!(graph_weights_vec, model.graph_weights)
end

SimPlot.plot_output_df(output_df)

In [ ]:
# for i in 120:180 # 協力率の急上昇局面
for i in 400:450 # 集団サイズの急収縮局面
    color_val_vec = [s == C ? colorant"blue" : colorant"red" for s in strategy_vec_vec[i]]
    g25 = SimpleGraph(graph_weights_vec[i] .>= 0.25)
    g50 = SimpleGraph(graph_weights_vec[i] .>= 0.50)
    g75 = SimpleGraph(graph_weights_vec[i] .>= 0.75)

    println("第$(i)世代")
    println("協力率: $(mean(strategy_vec_vec[i] .== C))")
    g25_components = [length(c) for c in connected_components(g25)]
    g50_components = [length(c) for c in connected_components(g50)]
    g75_components = [length(c) for c in connected_components(g75)]
    println("弱い繋がりグループ: $(length(g25_components))個, $(g25_components)")
    println("中程度の繋がりグループ: $(length(g50_components))個, $(g50_components)")
    println("強い繋がりグループ: $(length(g75_components))個, $(g75_components)")

    display(hstack(
        gplot(g25, nodefillc=color_val_vec, spring_layout(g25, C = 4.0)...),
        gplot(g50, nodefillc=color_val_vec, spring_layout(g50, C = 4.0)...),
        gplot(g75, nodefillc=color_val_vec, spring_layout(g75, C = 4.0)...)
    ))

    model.graph_weights = Simulation.convert_to_2nd_order_weights(graph_weights_vec[i])
    Simulation.normalize_graph_weights!(model)
    g25 = SimpleGraph(model.graph_weights .>= 0.25)
    g50 = SimpleGraph(model.graph_weights .>= 0.50)
    g75 = SimpleGraph(model.graph_weights .>= 0.75)
    g25_components = [length(c) for c in connected_components(g25)]
    g50_components = [length(c) for c in connected_components(g50)]
    g75_components = [length(c) for c in connected_components(g75)]
    println("弱い繋がりグループ: $(length(g25_components))個, $(g25_components)")
    println("中程度の繋がりグループ: $(length(g50_components))個, $(g50_components)")
    println("強い繋がりグループ: $(length(g75_components))個, $(g75_components)")

    display(hstack(
        gplot(g25, nodefillc=color_val_vec, spring_layout(g25, C = 4.0)...),
        gplot(g50, nodefillc=color_val_vec, spring_layout(g50, C = 4.0)...),
        gplot(g75, nodefillc=color_val_vec, spring_layout(g75, C = 4.0)...)
    ))
end

In [ ]:
@show maximum(graph_weights)
@show minimum(graph_weights)

In [ ]:
G_0 = SimpleGraph(graph_weights .>= 0.0)
G_1 = SimpleGraph(graph_weights .>= 0.1)
G_2 = SimpleGraph(graph_weights .>= 0.2)
G_4 = SimpleGraph(graph_weights .>= 0.4)
G_5 = SimpleGraph(graph_weights .>= 0.5)
G_6 = SimpleGraph(graph_weights .>= 0.6)
G_8 = SimpleGraph(graph_weights .>= 0.8)
G_9 = SimpleGraph(graph_weights .>= 0.9)
G = SimpleWeightedGraph(graph_weights);

In [ ]:
@show components = connected_components(G_4)
@show component_count = length(components)
colors = distinguishable_colors(component_count)
display(colors)

In [ ]:
get_componets(g::SimpleGraph)::Vector{Int} =
    sort([length(c) for c in connected_components(g)], rev = true)

plot_graph(g::SimpleGraph) = gplot(g, spring_layout(g, C = 4.0)...);

In [ ]:
@show get_componets(G_0)
@show get_componets(G_1)
@show get_componets(G_2)
@show get_componets(G_4)
@show get_componets(G_5)
@show get_componets(G_6)
@show get_componets(G_8)
@show get_componets(G_9);

In [ ]:
plot_graph(G_4)

In [ ]:
plot_graph(G_8)

In [ ]:
# for _df in df_vec
#     p = SimPlot.plot_output_df(_df)
#     # savefig(p, "img/$(Dates.format(now(), "yyyymmdd_HHMMSSsss")).png")
#     display(p)
# end